In [1]:
import torch, time
import torch.nn as nn
import torch.optim as optim

from src.utils import *
from src.override_resnet import *


class Args:
    arch = 50
    dataset = "ImageNet"
    # dataset = "CIFAR100"
    lr = 0.001
    momentum = 0.9
    batch = 256
    epochs = 10
    save_every = 1
    quan = "static"
    only_eval = True
    verbose = True


args = Args()

In [2]:
def fuse_model(model) -> nn.Module:
    flag = False
    for m in model.modules():
        if m.__class__.__name__ == ResNet_quan.__name__:
            if flag == True:
                raise ValueError("ResNet_quan is already fused")
            flag = True
            torch.quantization.fuse_modules(
                m,
                ["conv1", "bn1", "relu"],
                inplace=True,
            )

        if type(m) == BottleNeck_quan:
            torch.quantization.fuse_modules(
                m,
                [
                    ["conv1", "bn1", "relu1"],
                    ["conv2", "bn2", "relu2"],
                    ["conv3", "bn3"],
                ],
                inplace=True,
            )
            if m.downsample is not None:
                torch.quantization.fuse_modules(
                    m.downsample,
                    ["0", "1"],
                    inplace=True,
                )
    return model

In [3]:
# %% my code

args = Args()
# %% Load the ResNet-50 model
if args.quan == "fp32":
    # case 0 : no quantization case
    print("----------No quantization enabled")
    device = str(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
    model = layers_mapping[args.arch](
        weights=pretrained_weights_mapping[args.arch]
    ).to(device)

elif args.quan == "dynamic":
    # case 1 : Dynamic Quantization
    print("----------Dynamic Quantization enabled")
    device = "cuda"
    model = resnet50_quan(weights=pretrained_weights_mapping[args.arch]).to(device)
    quantized_model = torch.quantization.quantize_dynamic(
        model, {torch.nn.Linear}, dtype=torch.qint8
    )
    model = quantized_model

elif args.quan == "static":
    # case 2 : Static Quantization
    print("----------Static Quantization enabled")
    device = "cpu"
    model = resnet50_quan(weights=pretrained_weights_mapping[args.arch]).to(device)

elif args.quan == "qat":
    # case 3 : Quantization Aware Training
    print("----------Quantization Aware Training enabled")
else:
    raise ValueError("Invalid quantization method")

_folder_path = f"resnet{args.arch}_{args.dataset}" + "_" + args.quan
_file_name = (
    f"resnet{args.arch}_{args.dataset}_epoch"  # resnet18_cifar10_epoch{epoch}.pth
)

# %%Set up training and evaluation processes
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
train_loader, test_loader = GetDataset(
    dataset_name=args.dataset,
    device=device,
    root="data",
    batch_size=args.batch,
)
print(device)

----------Static Quantization enabled
cuda


# 0. REF

In [4]:
model.eval()
start_time = time.time()
eval_loss, eval_acc = SingleEpochEval(model, test_loader, criterion, device)
end_time = time.time()
print_size_of_model(model)
print("Ref Model : ", end_time - start_time)
ref_time = end_time - start_time
print(f"Eval Loss: {eval_loss:.4f}, Eval Acc: {eval_acc:.2f}%")
print(model.layer1[0])

100%|██████████| 196/196 [00:45<00:00,  4.27it/s]


Size (MB): 102.53111
Ref Model :  45.91718935966492 None
Eval Loss: 1.4145681584367946, Eval Acc: 80.344
BottleNeck_quan(
  (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (downsample): Sequential(
    (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu1): ReLU()
  (relu2): ReLU()
  (relu3): ReLU()
  (add): FloatFunctional(
    (activation_post_process): Identity()
  )
)


# fuse 확인

In [5]:
model = fuse_model(model)
print(print_size_of_model(model))
print(model.layer1[0])

Size (MB): 102.160522
None
BottleNeck_quan(
  (conv1): ConvReLU2d(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (bn1): Identity()
  (conv2): ConvReLU2d(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (bn2): Identity()
  (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
  (bn3): Identity()
  (relu): ReLU(inplace=True)
  (downsample): Sequential(
    (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    (1): Identity()
  )
  (relu1): Identity()
  (relu2): Identity()
  (relu3): ReLU()
  (add): FloatFunctional(
    (activation_post_process): Identity()
  )
)


준비

In [6]:
model.qconfig = torch.quantization.default_qconfig
print(model.qconfig)
torch.quantization.prepare(model, inplace=True)
print("Post Training Quantization Prepare: Inserting Observers")

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, quant_min=0, quant_max=127){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})
Post Training Quantization Prepare: Inserting Observers


calibration

In [7]:
eval_loss, eval_acc = SingleEpochEval(model, test_loader, criterion, device)
print(f"Eval Loss: {eval_loss:.4f}, Eval Acc: {eval_acc:.2f}%")
print("Post Training Quantization: Calibration done")

100%|██████████| 196/196 [00:46<00:00,  4.23it/s]

Eval Loss: 1.4145323558121312, Eval Acc: 80.344
Post Training Quantization: Calibration done


convert

In [8]:
torch.quantization.convert(model, inplace=True)
print("Post Training Quantization: Convert done")

Post Training Quantization: Convert done


# Static quantization 완료

In [9]:
start_time = time.time()
eval_loss, eval_acc = SingleEpochEval(model, test_loader, criterion, device)
end_time = time.time()
quan_time = end_time - start_time
print_size_of_model(model)
print("Quantized Model : ", end_time - start_time)
print(f"Eval Loss: {eval_loss:.4f}, Eval Acc: {eval_acc:.2f}%")
print("Post Training Quantization: Eval done")

  0%|          | 0/196 [00:03<?, ?it/s]


RuntimeError: getCudnnDataTypeFromScalarType() not supported for QUInt8

In [10]:
print(f"Ref Model : {ref_time:.2f}ms, Quantized Model : {quan_time:.2f}ms, Speedup : {ref_time/quan_time:.2f}x")

NameError: name 'quan_time' is not defined